# NFA to DFA

In [ ]:
from collections import defaultdict
from graphviz import Digraph

class DFA:
    def __init__(self, states, alphabet, transitions, start_state, accepting_states):
        self.states = states
        self.alphabet = alphabet
        self.transitions = transitions
        self.start_state = start_state
        self.accepting_states = accepting_states

def epsilon_closure(states):
    stack = list(states)
    closure = set(states)

    while stack:
        state = stack.pop()
        if 'ε' in state.transitions:
            for next_state in state.transitions['ε']:
                if next_state not in closure:
                    closure.add(next_state)
                    stack.append(next_state)

    return closure

def move(states, symbol):
    result = set()
    for state in states:
        if symbol in state.transitions:
            result.update(state.transitions[symbol])
    return result

def nfa_to_dfa(nfa):
    alphabet = set(sym for state in nfa.states for sym in state.transitions if sym != 'ε')

    start_closure = frozenset(epsilon_closure({nfa.starting_state}))

    dfa_states = {start_closure: 'D0'}
    dfa_transitions = {}
    unprocessed_states = [start_closure]
    accepting_states = set()

    while unprocessed_states:
        current_set = unprocessed_states.pop()
        current_name = dfa_states[current_set]

        dfa_transitions[current_name] = {}

        if any(state.accepting for state in current_set):
            accepting_states.add(current_name)

        for symbol in alphabet:
            next_states = epsilon_closure(move(current_set, symbol))

            if not next_states:
                continue

            if next_states not in dfa_states:
                new_name = f"D{len(dfa_states)}"
                dfa_states[next_states] = new_name
                unprocessed_states.append(next_states)

            dfa_transitions[current_name][symbol] = dfa_states[next_states]

    return DFA(states=set(dfa_states.values()),
               alphabet=alphabet,
               transitions=dfa_transitions,
               start_state='D0',
               accepting_states=accepting_states)

def visualize_dfa(dfa):
    dot = Digraph(format='png')
    dot.attr(rankdir='LR')

    dot.node('start', shape='point')
    dot.edge('start', dfa.start_state)

    for state in dfa.states:
        shape = 'doublecircle' if state in dfa.accepting_states else 'circle'
        dot.node(state, shape=shape)

        if state in dfa_transitions:
            for symbol, target in dfa.transitions[state].items():
                dot.edge(state, target, label=symbol)

    dot.render('dfa_visualization', view=True)

# Example usage
postfix_regex = "ab"
nfa = construct_nfa(postfix_regex)
dfa = nfa_to_dfa(nfa)
visualize_dfa(dfa)


In [20]:
import json
import graphviz
from collections import defaultdict, deque

def load_nfa_from_json(json_file):
    with open(json_file, 'r') as f:
        return json.load(f)

def epsilon_closure(states, nfa):
    closure = set(states)
    stack = list(states)

    while stack:
        current = stack.pop()
        if 'ε' in nfa.get(current, {}):
            for next_state in nfa[current]['ε']:
                if next_state not in closure:
                    closure.add(next_state)
                    stack.append(next_state)

    return closure

def move(states, symbol, nfa):
    next_states = set()
    for state in states:
        if symbol in nfa[state]:
            next_states.update(nfa[state][symbol])
    return next_states

def nfa_to_dfa(nfa_json):
    start_state = nfa_json['startingState']

    # Extract NFA transitions
    nfa = {state: {k: v if isinstance(v, list) else [v] for k, v in transitions.items() if k != 'isTerminatingState'} for state, transitions in nfa_json.items() if state != 'startingState'}

    start_closure = epsilon_closure({start_state}, nfa)

    dfa_transitions = {}
    dfa_start = '_'.join(sorted(start_closure))
    dfa_accepting_states = set()

    queue = deque([start_closure])
    visited = {frozenset(start_closure): dfa_start}

    while queue:
        current_set = queue.popleft()
        current_label = visited[frozenset(current_set)]
        dfa_transitions[current_label] = {}

        # Check if any NFA state is accepting
        if any(nfa_json[state].get('isTerminatingState', False) for state in current_set):
            dfa_accepting_states.add(current_label)

        # Get all possible input symbols
        symbols = set(symbol for state in current_set for symbol in nfa[state] if symbol != 'ε')

        for symbol in symbols:
            next_states = epsilon_closure(move(current_set, symbol, nfa), nfa)
            if not next_states:
                continue

            next_label = '_'.join(sorted(next_states))

            if frozenset(next_states) not in visited:
                visited[frozenset(next_states)] = next_label
                queue.append(next_states)

            dfa_transitions[current_label][symbol] = next_label

    return dfa_transitions, dfa_start, dfa_accepting_states

def visualize_dfa(dfa_transitions, dfa_start, dfa_accepting_states):
    dot = graphviz.Digraph(format='png', graph_attr={'rankdir': 'LR'})

    dot.node('', shape='none')
    dot.edge('', dfa_start)

    for state, transitions in dfa_transitions.items():
        shape = 'doublecircle' if state in dfa_accepting_states else 'circle'
        dot.node(state, shape=shape)

        for symbol, next_state in transitions.items():
            dot.edge(state, next_state, label=symbol)

    dot.render('dfa_output', view=True)

def save_dfa_to_json(dfa_transitions, dfa_start, dfa_accepting_states, output_file):
    dfa_json = {
        "startingState": dfa_start
    }

    for state, transitions in dfa_transitions.items():
        dfa_json[state] = {
            "isTerminatingState": state in dfa_accepting_states
        }
        for symbol, next_state in transitions.items():
            dfa_json[state][symbol] = next_state

    with open(output_file, 'w') as f:
        json.dump(dfa_json, f, indent=4, ensure_ascii=False)  # Prevent Unicode escaping

    print(f"DFA saved to {output_file}")

# Load NFA JSON
nfa_json_file = 'nfa.json'  # Replace with your NFA JSON file path
nfa_json = load_nfa_from_json(nfa_json_file)

# Convert NFA to DFA and visualize
dfa_transitions, dfa_start, dfa_accepting_states = nfa_to_dfa(nfa_json)
visualize_dfa(dfa_transitions, dfa_start, dfa_accepting_states)

# Save the DFA to a JSON file
save_dfa_to_json(dfa_transitions, dfa_start, dfa_accepting_states, 'dfa_output.json')


DFA saved to dfa_output.json


# Cleaned Code

In [ ]:
class DFA:
    def __init__(self):
        self.states = {}
        self.startingState = None
        self.transitions = {}
        self.acceptingStates = set()

    def add_state(self, state, is_accepting=False):
        self.states[state.name] = state
        if is_accepting:
            self.acceptingStates.add(state.name)

    def add_transition(self, from_state, to_state, symbol):
        if from_state.name not in self.transitions:
            self.transitions[from_state.name] = {}
        self.transitions[from_state.name][symbol] = to_state.name
class NFAtoDFAConverter:
    def epsilon_closure(self,state,NFA):
        closure = {state}
        stack = [state]

        while stack:
            current = stack.pop()

            if "ε" in NFA[current]:
                for target in NFA[current]["ε"]:
                    if target not in closure:
                        closure.add(target)
                        stack.append(target)

        return closure

